# pandas高级应用

# 分类数据

In [1]:
import numpy as np

In [2]:
import pandas as pd

## 背景和目的

In [3]:
values=pd.Series(['apple','orange','apple','apple']*2)

In [4]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [5]:
pd.unique(values)
# 提取出不同的值

array(['apple', 'orange'], dtype=object)

In [14]:
pd.value_counts(values)
# 计算频率

0    6
1    2
dtype: int64

In [9]:
values=pd.Series([0,1,0,0]*2)

In [10]:
dim=pd.Series(['apple','orange'])

In [11]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

## pandas的分类类型

In [12]:
fruits=['apple','orange','apple','apple']*2

In [13]:
N=len(fruits)

In [17]:
df=pd.DataFrame({'fruit':fruits,
              'basket_id':np.arange(N),
             'count':np.random.randint(3,15,size=N),
             'weight':np.random.uniform(0,4,size=N)},
            columns=['basket_id','fruit','count','weight'])

In [18]:
df

,basket_id,fruit,count,weight
0,0,apple,9,1.436118
1,1,orange,12,2.074674
2,2,apple,6,0.953131
3,3,apple,4,3.953589
4,4,apple,13,2.049119
5,5,orange,5,1.243412
6,6,apple,12,1.509649
7,7,apple,14,3.187523


In [20]:
fruit_cat=df['fruit'].astype('category')

# df['fruit']=df['fruit'].astype('category')

In [21]:
fruit_cat
# 此时不是一个Numpy数组，而是一个pandas.Categorical

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [22]:
c=fruit_cat.values

In [23]:
type(c)

pandas.core.arrays.categorical.Categorical

In [24]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [25]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [ ]:
# 列通过分配 转换结果，转化为分类

In [26]:
df['fruit']=df['fruit'].astype('category')

# 使得列转化为cat

In [27]:
df.fruit
# 

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [28]:
my_cat=pd.Categorical(['foo','bar','baz','foo','bar'])

# pd.categorical

In [29]:
my_cat

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [ ]:
# 获得了分类编码，还可以使用from_codes构造器

In [33]:
cat=['foo','bar','baz']

In [31]:
codes=[0,1,2,0,0,1]

In [34]:
my_cat_2=pd.Categorical.from_codes(codes,cat)

In [35]:
my_cat_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [37]:
ordered_cat=pd.Categorical.from_codes(codes,cat,
                                      ordered=True)

In [38]:
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [39]:
my_cat_2.as_ordered()
# 无序的分类可以通过as_ordered()排序

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

## 用分类进行计算

In [40]:
np.random.seed(12345)

In [43]:
draws=np.random.randn(1000)
draws[:5]

array([-0.57882769,  1.84744626,  0.45373927, -0.30245024,  1.40255769])

In [44]:
bins=pd.qcut(draws,4)

In [45]:
bins

[(-0.707, 0.0153], (0.716, 2.654], (0.0153, 0.716], (-0.707, 0.0153], (0.716, 2.654], ..., (0.716, 2.654], (0.0153, 0.716], (0.0153, 0.716], (0.716, 2.654], (0.716, 2.654]]
Length: 1000
Categories (4, interval[float64]): [(-3.429, -0.707] < (-0.707, 0.0153] < (0.0153, 0.716] < (0.716, 2.654]]

In [46]:
bins=pd.qcut(draws,4,labels=['Q1','Q2','Q3','Q4'])

In [47]:
bins

[Q2, Q4, Q3, Q2, Q4, ..., Q4, Q3, Q3, Q4, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [49]:
bins.codes[:10]

array([1, 3, 2, 1, 3, 3, 0, 1, 2, 3], dtype=int8)

In [50]:
bins=pd.Series(bins,name='quartile')

In [55]:
results=(pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index())

In [56]:
# 

In [57]:
results

,quartile,count,min,max
0,Q1,250,-3.428254,-0.710197
1,Q2,250,-0.705960,0.014029
2,Q3,250,0.016520,0.715103
3,Q4,250,0.718557,2.653656


In [58]:
# bins 命名为quartile  用bins进行分类   agg 为需要的变量   reset_index()

## 用分类提高性能

In [60]:
N=10000000

In [61]:
draws=pd.Series(np.random.randn(N))

In [62]:
labels=pd.Series(['foo','bar','baz','qux']*(N//4))

In [63]:
cat=labels.astype('category')

In [64]:
labels.memory_usage()

80000080

In [65]:
cat.memory_usage()

10000272

In [66]:
%time_=labels.astype('category')

UsageError: Line magic function `%time_=labels.astype('category')` not found.


In [67]:
## 分类方法

In [68]:
s=pd.Series(['a','b','c','d']*2)

In [69]:
cat_s=s.astype('category')

In [70]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [71]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [72]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [73]:
actual_categories=['a','b','c','d','e']

In [75]:
cat_s2=cat_s.cat.set_categories(actual_categories)

#set_categories()替换分类

In [76]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [77]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [78]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [79]:
#

In [80]:
cat_s3=cat_s[cat_s.isin(['a','b'])]


In [81]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [83]:
cat_s3.cat.remove_unused_categories()
# 删除没看到的分类

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

## 为建模创建虚拟变量

In [85]:
# one-hot编码

In [86]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [87]:
pd.get_dummies(cat_s)
# get_dummies() 转化为虚拟变量df

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## GroupBy高级应用

In [89]:
# apply agg

In [90]:
# transform

In [91]:
df=pd.DataFrame({'key':['a','b','c']*4,
                'value':np.arange(12.)})

In [92]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [94]:
g=df.groupby('key').value

In [95]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [97]:
g.transform(lambda x:x.mean())

# 传递函数  lambda x :x.mean()

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [98]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [99]:
g.transform(lambda x :x*2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [100]:
g.transform( lambda x :x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [121]:
def normalize(x):
    return (x - x.mean()) / x.std()
# 简单聚合构造的分组转换函数


In [109]:
normalized=(df['value']-g.transform('mean'))/g.transform('std')

In [110]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [122]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [123]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [124]:
# 先def normalize  后计算

## 分组的时间重采样

In [126]:
N=15

In [129]:
times=pd.date_range('2017-5-20 00:00',freq='1min',periods=N)

# 频率为1分钟

In [131]:
df=pd.DataFrame({'time':times,
               'value':np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [133]:
df.set_index('time').resample('5min').count()

# 用time做索引

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [134]:
df2=pd.DataFrame({'time':times.repeat(3),
                 'key':np.tile(['a','b','c'],N),
                 'value':np.arange(N*3.)})

In [135]:
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [136]:
time_key=pd.TimeGrouper('5min')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """Entry point for launching an IPython kernel.


In [137]:
resampled=(df2.set_index('time').groupby(['key',time_key]).sum())

In [138]:
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [139]:
resampled.reset_index()

# reset_index()还原索引
# set_index（key） 设置key为索引

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


# 链式编程技术

In [167]:
N=8
df=pd.DataFrame({'fruit':fruits,
              'basket_id':np.arange(N),
             'count':np.random.randint(3,15,size=N),
             'weight':np.random.uniform(0,4,size=N)},
            columns=['basket_id','fruit','count','weight'])

In [144]:
df

,basket_id,fruit,count,weight
0,0,apple,8,0.170217
1,1,orange,13,1.232264
2,2,apple,12,0.021810
3,3,apple,14,0.049770
4,4,apple,3,0.901098
5,5,orange,14,0.733271
6,6,apple,13,1.292710
7,7,apple,4,0.431056


In [145]:
df.set_index('basket_id')

,fruit,count,weight
basket_id,,,
0,apple,8,0.170217
1,orange,13,1.232264
2,apple,12,0.021810
3,apple,14,0.049770
4,apple,3,0.901098
5,orange,14,0.733271
6,apple,13,1.292710
7,apple,4,0.431056


In [148]:
df2=df[df['count']>10]

In [150]:
df2

,basket_id,fruit,count,weight
1,1,orange,13,1.232264
2,2,apple,12,0.021810
3,3,apple,14,0.049770
5,5,orange,14,0.733271
6,6,apple,13,1.292710


In [151]:
df2['new']=df2['count']-df2['count'].mean()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [152]:
df2

,basket_id,fruit,count,weight,new
1,1,orange,13,1.232264,-0.2
2,2,apple,12,0.021810,-1.2
3,3,apple,14,0.049770,0.8
5,5,orange,14,0.733271,0.8
6,6,apple,13,1.292710,-0.2


In [153]:
result=df2.groupby('fruit').new.std()

In [154]:
result

fruit
apple     1.000000
orange    0.707107
Name: new, dtype: float64

In [155]:
df2=df.copy()

In [158]:
df2['count']=3
df2

,basket_id,fruit,count,weight
0,0,apple,3,0.170217
1,1,orange,3,1.232264
2,2,apple,3,0.021810
3,3,apple,3,0.049770
4,4,apple,3,0.901098
5,5,orange,3,0.733271
6,6,apple,3,1.292710
7,7,apple,3,0.431056


In [159]:
df2=df.assign(count=3)
# 浅拷贝

In [160]:
df2

,basket_id,fruit,count,weight
0,0,apple,3,0.170217
1,1,orange,3,1.232264
2,2,apple,3,0.021810
3,3,apple,3,0.049770
4,4,apple,3,0.901098
5,5,orange,3,0.733271
6,6,apple,3,1.292710
7,7,apple,3,0.431056


In [161]:
### method chaining

In [172]:
result=(df2.assign(new=df2['count']-df2['count'].mean())).groupby('fruit').new.std()

In [173]:
result

fruit
apple     0.0
orange    0.0
Name: new, dtype: float64

## 管道方法

In [174]:
# pipe